# How to replace the noun_chunks generator in spaCy
    * noun_chunks are generated by `syntax_iterators.py` in spaCy/lang/<lang>` = hardcoded file
    * noun_chunks are neede for pytextrank which extracts keywords
    * this code shows how to replace the original noun_chunks generator with custom function
    * see issue #54 in pytextrank for further info: https://github.com/DerwenAI/pytextrank/issues/54
    

In [2]:
import spacy_udpipe, spacy, pytextrank
from spacy.matcher import Matcher
from spacy.attrs import POS

def get_chunks(doc):
    np_label = doc.vocab.strings.add("NP")
    matcher = Matcher(nlp.vocab)
    pattern = [{POS: 'ADJ', "OP": "+"}, {POS: {"IN": ["NOUN", "PROPN"]}, "OP": "+"}]
    matcher.add("Adjective(s), (p)noun", None, pattern)
    matches = matcher(doc)

    for match_id, start, end in matches:
        yield start, end, np_label
        
spacy_udpipe.download("sk") # download model
nlp = spacy_udpipe.load("sk")
nlp.Defaults.syntax_iterators = {"noun_chunks" : get_chunks}  #noun_chunk replacement
#spacy.lang.sk.Slovak.Defaults.syntax_iterators

tr = pytextrank.TextRank(logger=None)
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

text = "Wikipédia je webová encyklopédia s otvoreným obsahom, ktorú možno slobodne čítať aj upravovať. Je sponzorovaná neziskovou organizáciou Wikimedia Foundation. Má 285 nezávislých jazykových vydaní vrátane slovenského a najrozsiahlejšieho anglického. Popri článkoch encyklopedického typu obsahuje, najmä anglická encyklopédia, aj články podobajúce sa almanachu, atlasu či stránky aktuálnych udalostí. Wikipédia je jedným z najpopulárnejších zdrojov informácií na webe s približne 13 miliardami zobrazení mesačne. Jej rast je skoro exponenciálny. Wikipédii (takmer 2 milióny). Wikipédia bola spustená 15. januára 2001 ako doplnok k expertmi písanej Nupedii. So stále rastúcou popularitou sa Wikipédia stala podhubím pre sesterské projekty ako Wikislovník (Wiktionary), Wikiknihy (Wikibooks) a Wikisprávy (Wikinews). Jej články sú upravované dobrovoľníkmi vo wiki štýle, čo znamená, že články môže meniť v podstate hocikto. Wikipediáni presadzujú politiku „nestranný uhol pohľadu“. Podľa nej relevantné názory ľudí sú sumarizované bez ambície určiť objektívnu pravdu. Vzhľadom na to, že Wikipédia presadzuje otvorenú filozofiu, jej najväčším problémom je vandalizmus a nepresnosť. "
doc = nlp(text)

print("Noun chunks:")
for nc in doc.noun_chunks:
    print(nc)
    
print("\nKeywords:")
for phrase in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(phrase.rank, phrase.count, phrase.text))
    print(phrase.chunks)    

Already downloaded a model for the 'sk' language
Noun chunks:
webová encyklopédia
otvoreným obsahom
neziskovou organizáciou
sponzorovaná neziskovou organizáciou
jazykových vydaní
nezávislých jazykových vydaní
encyklopedického typu
anglická encyklopédia
aktuálnych udalostí
najpopulárnejších zdrojov
najpopulárnejších zdrojov informácií
písanej Nupedii
rastúcou popularitou
sesterské projekty
upravované dobrovoľníkmi
nestranný uhol
nestranný uhol pohľadu
relevantné názory
relevantné názory ľudí
objektívnu pravdu
otvorenú filozofiu
najväčším problémom

Keywords:
0.0793     1  anglická encyklopédia
[anglická encyklopédia]
0.0791     1  otvorenú filozofiu
[otvorenú filozofiu]
0.0784     1  nezávislých jazykových vydaní
[nezávislých jazykových vydaní]
0.0770     1  sponzorovaná neziskovou organizáciou
[sponzorovaná neziskovou organizáciou]
0.0765     1  najpopulárnejších zdrojov informácií
[najpopulárnejších zdrojov informácií]
0.0753     1  webová encyklopédia
[webová encyklopédia]
0.0738    